# 使用sklearning决策树工具对watermelon数据进行分类训练

In [308]:
import csv
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing
from sklearn import tree

wame_data = open('watermelon_4_3.csv','rt')     ##打开一个IO file对象
reader = csv.reader(wame_data)                  #file对象wame_data作为参数传递给csv.reader方法，获得一个csv.reader对象
#print(reader)

#读取表头
headers = next(reader)                      
#print(headers)
headers1 = sorted(headers[1:-1])        
#查看特征名称排序后的情况，后续会用到DictVectorizer函数功能会把字符型特征进行编码并按特征名称的字母顺序进行排序，便于对比观察。
#表头升序排序后赋值给headers1，sorted(headers[1:-1])不会影响headers原始内容。
#注意定界范围，heaers[1:-1]表示从第二个数据取到倒数第二个数据。
print(headers1)

#测试函数：用于查看数据集中每一列特征值有几种类型，如：color特征有3种类型：'dark_green', 'black', 'light_white'
def feature_kind(reader):
    for i in range(8):
    #del color_list[:]
    #color_set = set(color_list)
        color_list = []
        j=i+1
        wame_data.seek(0,0)   #调整数据指针到数据文件头
        next(reader)          #读一次数据，使数据指针指向第一行数据
        for row in reader:
            color_list.append(row[j])
        color_set = set(color_list)   #把列表转换为set集合，用于去重，可直观观察特征值的情况。
        print(color_set)
        print(len(color_set))
    wame_data.seek(0,0)   #调整数据指针到数据文件头
    next(reader)          #读一次数据，使数据指针指向第一行数据,恢复reader指针位置到函数调用前
    
feature_kind(reader)
    
#获得数据字典列表和标签表，以提供给DictVectorizer函数做特征向量变换的数据源。
#数据字典列表的格式：[{特征1：值1，特征2：值1，...，特征8：值1},{特征1：值2，特征2：值2，...，特征8：值2},...]

feature_list = []          #数据字典列表
result_list = []           #标签表

#把带数值类型的特征值转换为float类型，以便于在后续特征向量变换时保留数值特征。
for row in reader:
    result_list.append(row[-1])                  #生成标签表
    float_list = [float(i) for i in row[7:9]]   #把数字形状的str类型元素转换为float类型，以便于在特征向量变换时不受变换影响。
    com_list = row[1:7]+float_list
    #print(com_list)
    #feature_list.append(dict(zip(headers[1:-1],row[1:-1])))
    feature_list.append(dict(zip(headers[1:-1],com_list)))  #特征名称和特征值组合成字典列表
print(feature_list)

#特征向量变换，把描述型特征值变换为独热编码（one-hot），数值型特征值保留原值。
vec = DictVectorizer()          #把dict类型的list数据转换为numpy array
dummyX = vec.fit_transform(feature_list).toarray()

dummyY = result_list         #因为标签已经是二值化的，直接赋值即可
#dummyY = preprocessing.LabelBinarizer().fit_transform(result_list)  #使用preprocessing.LabelBinarizer().fit_transform()对标签进行二值化处理
#dummyY = [int(i) for i in result_list]     #因为标签已经是二值化的，元素转换为int类型，以便与str类型比较
#经测试，二值化【0，1】标签是str型和int型效果相同，即使使用preprocessing.LabelBinarizer().fit_transform()处理的效果也相同。
#因此，选择使用直接赋值方式

print(dummyX)
print(dummyY)

#设计子函数，用于把one-hot特征向量变换为二进制代码
def elementadd(code):
    code_i_item = ""
    for k in code:
        code_i_item = code_i_item + k
    return code_i_item

#测试函数：生成数据特征值与向量代码的对应关系，获得每个特征值与向量代码的对应表（方便预测数据的设计），
#如：['dark_green', '010'], ['black', '100'], ['light_white', '001']
def featurevalues_code_get():
    code_i = []
    feature_dict_add = {}
    feature_code_list = locals()
    for m in range(6):                            #初始化feature_code_list+"fm"，8个特征减去2个数值特征。
        feature_code_list["_f" + str(m)] = []
    #print(feature_code_list["_f" + str(m)])
    #特征名称-特征值字典列表排序，并删除特征值为数值的列表元素
    for i in range(len(feature_list)):
        feature_list_0_sort = sorted(feature_list[i].items(),key=lambda item:item[0])   #以特征名称对特征名称-特征值字典列表排序
        del feature_list_0_sort[5]               #删除sugar_rate对应的键值对
        del feature_list_0_sort[1]               #删除density对应的键值对
        #print(feature_list_0_sort)

    #遍历排序后的特征名称-特征值键值对数据，从中取出特征值并放入feature_i_item列表
        feature_i_item = []
        for k in feature_list_0_sort:       
            feature_i_item.append(k[1])
        print(feature_i_item)
        #feature_i.append(feature_i_element)

        #print(feature_i)

        #对dummyX进行处理，删除特征值为数值的元素，把特征向量转换为二进制代码，并得到特征值与特征向量二进制代码的去重组合列表。
        dummyX_i = list(dummyX[i])
        del dummyX_i[3]
        del dummyX_i[12]
        dummyX_i_item = list(map(str,map(int,dummyX_i)))
        #dummyX_i = list(map(str,map(int,dummyX[i])))
        #del dummyX_i[3]
        #del dummyX_i[12]
        #print(dummyX_i)
        code_i = []
        for j in range(0,12,3):
            code = dummyX_i_item[j:j+3]
            code_i.append(elementadd(code))

        code = dummyX_i_item[12:15]
        code_i.append(elementadd(code))
        code = dummyX_i_item[15:17]
        code_i.append(elementadd(code))
        print(code_i)
        #print(dummyX_i_item)
        for n in range(len(feature_i_item)):
            if [feature_i_item[n],code_i[n]] not in feature_code_list["_f" + str(n)]:
                feature_code_list["_f" + str(n)].append([feature_i_item[n],code_i[n]])

    #显示特征名称和“特征值-代码”对
    features = {"0":"color","1":"knocks","2":"navel","3":"root","4":"texture","5":"touch"}
    for m in range(len(feature_i_item)): 
        print(features[str(m)])
        print(feature_code_list["_f" + str(m)])

featurevalues_code_get()

#建立训练模型
clf = tree.DecisionTreeClassifier(criterion='entropy',random_state=0)
#调用特征list和标签list进行训练
clf = clf.fit(dummyX,dummyY)
print("训练模型clf："+ " "+str(clf))  

#生成决策树图
import pydotplus
dot_data = tree.export_graphviz(clf,feature_names=vec.get_feature_names(),filled=True,rounded=True,special_characters=True,out_file=None)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf("watermelon.pdf")

A = ([[0,1,0,0.293,0,0,1,1,0,0,1,0,0,0.042,1,0,0,1,0]])  
#颜色darkgreen,密度0.293,敲击light-heavily,瓜脐even（平），根蒂curl-up（卷），纹理blur（模糊），手感hard-smooth（硬滑）
B = ([[0,0,1,0.593,0,0,1,1,0,0,1,0,0,0.042,1,0,0,1,0]])
print(clf.predict(A))
print(clf.predict_proba(A))


['color', 'density', 'knocks', 'navel', 'root', 'sugar_ratio', 'texture', 'touch']
{'dark_green', 'black', 'light_white'}
3
{'little_curl_up', 'stiff', 'curl_up'}
3
{'heavily', 'little_heavily', 'clear'}
3
{'blur', 'little_blur', 'distinct'}
3
{'little_sinking', 'even', 'sinking'}
3
{'soft_stick', 'hard_smooth'}
2
{'0.556', '0.36', '0.593', '0.697', '0.657', '0.437', '0.343', '0.634', '0.403', '0.481', '0.608', '0.245', '0.639', '0.774', '0.719', '0.666', '0.243'}
17
{'0.149', '0.042', '0.103', '0.237', '0.198', '0.264', '0.057', '0.376', '0.318', '0.161', '0.215', '0.37', '0.267', '0.091', '0.211', '0.46', '0.099'}
17
[{'navel': 'sinking', 'touch': 'hard_smooth', 'density': 0.697, 'texture': 'distinct', 'root': 'curl_up', 'knocks': 'little_heavily', 'sugar_ratio': 0.46, 'color': 'dark_green'}, {'navel': 'sinking', 'touch': 'hard_smooth', 'density': 0.774, 'texture': 'distinct', 'root': 'curl_up', 'knocks': 'heavily', 'sugar_ratio': 0.376, 'color': 'black'}, {'navel': 'sinking', 'touch

In [307]:
B = ([[0,0,1,0.593,0,0,1,1,0,0,1,0,0,0.042,1,0,0,1,0]])
print(clf.predict(B))
print(clf.predict_proba(B))

['0']
[[1. 0.]]


In [280]:
wame_data = open('watermelon_4_3.csv','rt')     #打开一个IO file对象
reader = csv.reader(wame_data)                  #file对象wame_data作为参数传递给csv.reader方法，获得一个csv.reader对象
print(wame_data)
print(reader)
headers = next(reader)
headers = sorted(headers[1:-1])            #表头升序排序
print(headers1)

<_io.TextIOWrapper name='watermelon_4_3.csv' mode='rt' encoding='UTF-8'>
['color', 'density', 'knocks', 'navel', 'root', 'sugar_ratio', 'texture', 'touch']


In [293]:
X = [[0,0],[1,1]]
Y = [0,1]
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X,Y)

In [295]:
clf.predict([[3,3]])

array([1])

In [296]:
clf.predict_proba([[2,2]])

array([[0., 1.]])